In [20]:
import csv
import math
from sympy import *
import numpy as np
from scipy import special
from scipy.stats import binom, hypergeom
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
import random

   **entree:**
      r: longueur de la première rangée
      wi: poids de Hamming de la première rangée.
     **Sortie:**
      tableau numpy de la première rangée d'un bloc circulant

In [21]:
def genFirstRow(r, wi):

    firstRow = np.zeros(r, dtype=np.int32)
    # choisir les nombres wi de la plage (wi)
    randomArray = np.random.choice(r, wi, replace=False)

    for i in randomArray:
        firstRow[i] = 1

    return firstRow


**entrée:** la première ligne de la matrice circulante binaire. 
    **Sortie:** la matrice circulante binaire

In [22]:
def genCirculant(firstRow):
   
    rows = firstRow.size
    firstRow = np.array(firstRow, dtype = np.int32)
    M = np.zeros((rows, rows), dtype = np.int32)
    M[0,:] = firstRow

    for i in range(1,rows):
        for j in range(rows):
            M[i,j] = M[i-1, (j-1)%rows]
    return M

 **entrée:** la première ligne d une matrice circulante. 
    **sortie:** la première ligne de la transposée de la matrice circulante

In [23]:
def genTransposePoly(firstRow):
  
    transposePoly = np.zeros(len(firstRow), dtype = np.int32)
    transposePoly[0] = firstRow[0]
    transposePoly[1:] = np.flip(firstRow[1:])
    return transposePoly

**entrée:** la première rangée de 2 matrices circulantes. 
    **Sortie:** la première ligne du produit des matrices circulantes

In [24]:
def genProdPoly(firstRowA, firstRowB):

    r = len(firstRowA)
    prodPoly = np.zeros(r, dtype = np.int32)
    convolvePoly = np.convolve(firstRowA, firstRowB)

    if len(convolvePoly) > r:
        prodPoly[0:r] = convolvePoly[0:r]
        prodPoly[0: len(convolvePoly) - r] += convolvePoly[r:]
    else:
        prodPoly[0: len(convolvePoly)] = convolvePoly[0: len(convolvePoly)]
        
    return prodPoly

**entrée:** la première ligne d'une matrice circulante binaire. 
    **Sortie:**  la première ligne de l'inverse de la matrice circulante binaire

In [25]:
def genInvPoly(firstRow):
  
    r = len(firstRow)
    invPoly = np.zeros(r, dtype = np.int32)

    # convertir un tableau numpy en polynôme
    inv = convertNumpyToSympy(firstRow)

    #définir un anneau polynomial, F_2 / (x**r - 1)
    FField = "x**" + str(r) + "-" + str(1)
    FField = poly(FField, domain=FF(2))

    #recherche de l'inverse du polynôme dans l'anneau polynomial spécifié
    inv = invert(poly(inv, domain=FF(2)), poly(FField, domain=FF(2)))
    temp = convertSympyToNumpy(inv)

    invPoly[0 : len(temp)] = temp
    
    return invPoly

**entrée:** tableau d'entiers v. 
    **Sortie:** tableau d'entiers de v modulo 2

In [26]:
def convertBinary(v):
   
    for i in range(len(v)):
        v[i] = v[i] % 2

    return v

**entrée:** un tableau numpy.
    **sortie:** un polynôme Sympy
    **Hypothèses:** les coefficients sont non négatifs

In [27]:
def convertNumpyToSympy(f):
   
    polynomial = ""
    polynomial = polynomial + str(int(f[0]))
    
    for i in range(1, f.size):
        if f[i] != 0:
            polynomial += " + " + str(int(f[i])) + "*x**" + str(i)
    return polynomial

**entrée:** un polynôme Sympy. 
    **Sortie:** un tableau numpy

In [28]:
def convertSympyToNumpy(f):
    v = np.array(f.all_coeffs())

    #Sympy stocke les coefficients du polynôme f (x) en puissances décroissantes de x
    #ainsi l'ordre est inversé pour obtenir un tableau numpy de puissances croissantes de x
    v = np.flip(v)
    
    return v

   **Entree**:
    firstRow: la première ligne de la matrice QC-MDPC
    n: longueur du code / no QC-MDPC. de nœuds de bits dans le code QC-MDPC
    r: non. de lignes / cols de chaque bloc circulant du code QC-MDPC
    w: somme des poids (wi) de la première rangée de tous les blocs circulants.
    **Sortie**: une matrice QC-MDPC construite à partir de la première ligne
    notez que la distribution des poids est aléatoire.
    Notez que firstRow est de longueur n = n0 * r
    Définissez f_i (x) comme le polynôme de Hall de H_i

In [29]:
def genQCMDPC(n, r, w):
    n0 = int(n / r)
    wi = int(w / n0)
    
    H = np.zeros((r, r * n0), dtype = np.int32)
    H_i = np.zeros((r, r), dtype = np.int32)

    for i in range(n0):
        firstRow = genFirstRow(r, wi)
        H_i = genCirculant(firstRow)

        if i == 0:
            H = H_i
        else :
            H = np.concatenate((H, H_i), axis=1)

    filename = str(n) + "_" + str(r) + "_" + str(w) + "_" + "parityCheckMatrix.csv"

    np.savetxt(filename, H, delimiter = ",", fmt = "%d")

    return H


**Entrée:** matrice de contrôle de parité H.
    **Sortie:** matrice génératrice G correspondante

In [30]:
def genGenQCMDPC(H):
   
    r, n = H.shape
    n0 = int(n / r)
    H_i = np.zeros((r, r), dtype = np.int32)
    G = np.eye(n - r, dtype = np.int32)
    block0 = np.zeros((r, r), dtype = np.int32)

    #extraire les polynômes générateurs pour h_ {n_0-1} de H
    lastPoly = H[0, n - r : n]

    #calculer le polynôme générateur de l'inverse de h_ {n_0-1}
    invLastPoly = genInvPoly(lastPoly)

    #calculer le premier bloc circulant pour G
    temp = genProdPoly(invLastPoly, H[0, 0:r])
    temp1 = convertBinary(temp)
    temp2 = genTransposePoly(temp1)
    block0 = genCirculant(temp2)

    #calculer le bloc circulant suivant pour G et les concaténer
    for i in range(1, n0 - 1):
        temp = genProdPoly(invLastPoly, H[0, i*r : (i+1)*r])
        temp1 = convertBinary(temp)
        temp2 = genTransposePoly(temp1)
        block = genCirculant(temp2)
        block0 = np.concatenate((block0, block), axis = 0)

    if (n0 == 1):
        return G
    else :
     #concaténer la matrice d'identité d'ordre (n - r) et l'empilement de matrices circulantes pour former G
        G = np.concatenate((G, block0), axis = 1)

    w = sum(H[0, :])
    
    filename = str(n) + "_" + str(r) + "_" + str(w) + "_" + "GeneratorMatrix.csv"

    np.savetxt(filename, G, delimiter = ",", fmt = "%d")
    
    return G

In [34]:
# paramètres de code
n0 = 2
r = 10
wi = 7

#paramètres de décryptage
t = 13

d = r // 2

n = n0 * r
w = wi * n0

print('########## Generate keys ##########')
print("(r,d,t):", (r,wi,t))
# générer une matrice aléatoire (n, r, w) -QC-MDPC H
H = genQCMDPC(n, r, w)

# Générer la matrice génératrice G correspondante
G = genGenQCMDPC(H)

print("G:\n", G)


########## Generate keys ##########
(r,d,t): (10, 7, 13)
G:
 [[1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1]
 [0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1]
 [0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 1 1 1]
 [0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 1]]
